In [3]:
from selenium import webdriver
import time
import pandas as pd

In [4]:
driver = webdriver.Chrome('./chromedriver')
driver.maximize_window()
driver.implicitly_wait(10)

In [88]:
driver.get('https://basketball.realgm.com/international/league/4/Spanish-ACB/team/649/Acunsa-GBC')
driver.find_element_by_class_name('Frame-sc-1d4hofp-0.czVBsw').click()

In [31]:
#driver.delete_all_cookies()

In [89]:
uno = driver.find_elements_by_class_name('has-subnav.with-js')
dos = uno[5].click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".Frame-sc-1d4hofp-0.czVBsw"}
  (Session info: chrome=90.0.4430.72)


In [85]:
driver.find_element_by_xpath('//*[@id="site-takeover"]/div[3]/div/div[1]/div[5]/select').click()
driver.find_element_by_xpath('//*[@id="site-takeover"]/div[3]/div/div[1]/div[5]/select/option[6]').click()
driver.find_element_by_class_name('Frame-sc-1d4hofp-0.czVBsw').click()


ElementClickInterceptedException: Message: element click intercepted: Element <select onchange="open_network(this.options[this.selectedIndex].value)" class="ddl">...</select> is not clickable at point (200, 81). Other element would receive the click: <li class="has-subnav with-js">...</li>
  (Session info: chrome=90.0.4430.72)


### With this cell we get the name of each stat column.

In [62]:
tablita = driver.find_element_by_class_name("tablesaw.compact.tablesaw-swipe")
columnas = tablita.find_elements_by_tag_name("th")
interm = []
for a in columnas:
    interm.append(a)
columns_name = []
for name in interm: 
    columns_name.append(name.text)

### With this cell we get all the stats of every player in this season (2020-21).

In [78]:
equipos = driver.find_elements_by_class_name("has-subnav")

In [82]:
equipos = equipos[8:]
stats_list = []
for equipo in equipos:
    equipo.click()
    driver.find_element_by_class_name('Frame-sc-1d4hofp-0.czVBsw').click()
    driver.find_element_by_xpath('//*[@id="site-takeover"]/div[3]/div/div[1]/div[5]/select').click()
    driver.find_element_by_xpath('//*[@id="site-takeover"]/div[3]/div/div[1]/div[5]/select/option[6]').click()
    driver.find_element_by_class_name('Frame-sc-1d4hofp-0.czVBsw').click()
    tablita = driver.find_element_by_class_name("tablesaw.compact.tablesaw-swipe")
    players = tablita.find_elements_by_tag_name("tr")
    for player in players:
        stats = player.find_elements_by_tag_name("td")
        for stat in stats:
            stats_list.append(stat.text)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="site-takeover"]/div[3]/div/div[1]/div[5]/select"}
  (Session info: chrome=90.0.4430.72)
